In [4]:
import pandas as pd

# Cleaning up PG & E data 

In [111]:
# manipulating electrical data
edf = pd.read_csv("./data/electrical.csv")                    # read the data
edf['DATE'] = edf['DATE'].astype('datetime64')                # set the date type
edf['COST'] = edf['COST'].str.replace('$','')                 # remove unnecesary symbols
edf['COST'] = edf['COST'].astype('float64')                   # set the right type
edf = edf.set_index('DATE').resample('D').sum()               # Get the usage and cost total per day


# manipulating gas data
tmp = pd.read_csv("./data/gas.csv").drop(columns=['TYPE','UNITS','NOTES'])
gdf = pd.DataFrame()
gdf['GAS_DATE'] = tmp['DATE'].astype('datetime64')
tmp['COST'] = tmp['COST'].str.replace('$','')
gdf['GAS_COST'] = tmp['COST'].astype('float64')
gdf['GAS_USAGE'] = tmp['USAGE']

gdf = gdf.set_index('GAS_DATE')




data_combined = pd.concat([edf, gdf], axis=1, join='inner')     # join dataframes

data_combined.i

# data_combined.to_csv("data/pge_data.csv")                       # save data

,USAGE,COST,GAS_COST,GAS_USAGE
2014-07-30,0.48,0.00,0.00,0.00
2014-07-31,0.48,0.00,0.00,0.00
2014-08-01,0.48,0.00,0.00,0.00
2014-08-02,0.48,0.00,0.00,0.00
2014-08-03,0.48,0.00,0.00,0.00
...,...,...,...,...
2020-07-06,30.64,7.48,0.49,0.35
2020-07-07,55.99,13.64,0.49,0.35
2020-07-08,38.04,9.28,0.00,0.00
2020-07-09,0.00,0.00,0.00,0.00
